In [1]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK



__Maximize__ $$–4x_1 + 2x_2$$
__Subject To__
$$4x_1 + x_2 + x_3 = 20$$
$$2x_1 – x_2 ≥ 6$$
$$x_1 – x_2 + 5x_3 ≥ –5$$
$$–3x_1 + 2x_2 + x_3 ≤ 4$$ 

$$x_1 ≤ 0, x_2 ≥ 0, x_3\text{ unrestricted}$$


Production Requirements:

|Product|Period 1|Period 2|Period 3|Period 4|Period 5|
|-------|--------|--------|--------|--------|--------|
|Widgets|70      |125     |185     |190     |200     |
|Gadgets|200     |300     |295     |245     |240     |
|Flugels|140     |175     |205     |235     |230     |



In [20]:
# Creates a list of the Ingredients
periods = list(range(1,6))
products = ['widget', 'gadget', 'flugels']
ad_costs_list = [160, 120, 180]

ad_costs = dict(zip(products, ad_costs_list))

In [42]:
# Final
ads_variables = []
product_variables = []
# define variables

for period in periods:
    for product in products:
        ads_variables.append(f"{product}_advertising_investment_p{period}")
        product_variables.append(f"{product}_produced_in_p{period}")

In [43]:
ads_variables

['widget_advertising_investment_p1',
 'gadget_advertising_investment_p1',
 'flugels_advertising_investment_p1',
 'widget_advertising_investment_p2',
 'gadget_advertising_investment_p2',
 'flugels_advertising_investment_p2',
 'widget_advertising_investment_p3',
 'gadget_advertising_investment_p3',
 'flugels_advertising_investment_p3',
 'widget_advertising_investment_p4',
 'gadget_advertising_investment_p4',
 'flugels_advertising_investment_p4',
 'widget_advertising_investment_p5',
 'gadget_advertising_investment_p5',
 'flugels_advertising_investment_p5']

The M&F Department projects that each \\$160 invested in advertising Widgets in a particular period creates additional demand for one Widget in the next period. The corresponding values for Gadgets and Flugels are \\$120 and \\$180, respectively. However, the total advertising budget is limited to \\$70,000 for the entire planning horizon.

Throughout the planning horizon, NU Industries will sell: 
* Widgets for \\$2490 
* Gadgets for \\$1990 
* Flugels for \\$2970. 

The products can be manufactured at either of NU's Manufacturing Plants.


In [ ]:
[ad_costs[i] * ingredient_vars[i] for product in products]

In [44]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
advertising_vars = LpVariable.dicts("", ads_variables, 0)
product_vars = LpVariable.dicts("", product_variables, 0)
    
    
# defines the problem
nu_industries = LpProblem("NU Industries", LpMaximize)

In [45]:
advertising_vars

{'widget_advertising_investment_p1': _widget_advertising_investment_p1,
 'gadget_advertising_investment_p1': _gadget_advertising_investment_p1,
 'flugels_advertising_investment_p1': _flugels_advertising_investment_p1,
 'widget_advertising_investment_p2': _widget_advertising_investment_p2,
 'gadget_advertising_investment_p2': _gadget_advertising_investment_p2,
 'flugels_advertising_investment_p2': _flugels_advertising_investment_p2,
 'widget_advertising_investment_p3': _widget_advertising_investment_p3,
 'gadget_advertising_investment_p3': _gadget_advertising_investment_p3,
 'flugels_advertising_investment_p3': _flugels_advertising_investment_p3,
 'widget_advertising_investment_p4': _widget_advertising_investment_p4,
 'gadget_advertising_investment_p4': _gadget_advertising_investment_p4,
 'flugels_advertising_investment_p4': _flugels_advertising_investment_p4,
 'widget_advertising_investment_p5': _widget_advertising_investment_p5,
 'gadget_advertising_investment_p5': _gadget_advertising

In [39]:
# Objective Function
nu_industries += (
(2490 * product_vars['widget_produced_in_period_1']) - (advertising_vars['widget_advertising_period_1']) + 
(2490 * (product_vars['widget_produced_in_period_2'] + advertising_vars['widget_advertising_period_1'])) - 
    (advertising_vars['widget_advertising_period_2'] * 160)
)

/Users/robords/.venv/msds460_final/lib/python3.9/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [40]:
# The problem data is written to an .lp file
nu_industries.writeLP("NU_Industries.lp")

[_widget_advertising_period_1,
 _widget_advertising_period_2,
 _widget_produced_in_period_1,
 _widget_produced_in_period_2]

In [2]:
# defines the problem
nu_industries = LpProblem("NU Industries", LpMaximize)

# define objective function - Maximize profit
prob2 += 300*cars + 400*trucks - (type_1_machines*50)

# define constraints
prob2 += (.8*cars + trucks - type_1_machines <= 0, 'Cars and trucks take up type 1 machine time')
prob2 += (type_1_machines <= 98, 'Carco can rent up to 98 Type 1 machines')
prob2 += (.6*cars + .7*trucks<= 73, 'Carco now has 73 Type 2 machines')
prob2 += (2*cars + 3*trucks <= 260, 'The company has 260 tons of steel available')
prob2 += (cars >= 88, 'Marketing considerations dictate that at least 88 cars be produced')
prob2 += (trucks >= 26, 'Marketing considerations dictate that at least 26 trucks be produced')

# solve the problem
prob2.writeLP("prob2.lp")
prob2.solve(GLPK(msg=0, options=['--ranges', 'homework_2_prob2.sen']))
print ("Status:", LpStatus[prob2.status])

# Note, we are only able to get sensitivity information because we are solving
# as a linear program.  If we solved as an Integer Program, then no 
# sensitivity information would be available.
print ("Objective", value(prob2.objective))
for v in prob2.variables():
    print(v.name, "=", v.varValue)
print ("")


Status: Optimal
Objective 32540.0
cars = 88.0
trucks = 27.6
type_1_machines = 98.0



/Users/robords/.venv/msds460_final/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
